# What is the trend and factors influencing the enrolment in undergraduate programs at LSE? 


## Data Acquisition

### data from LSE course calendar for each course over the years -each year is a diff df

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_course_numbers(url, year):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    content = soup.find("div", attrs={"class": "right-container"})
    links = content.find_all("a", href=True)
    course_list = []
    for link in links:
        text = link.text.strip()
        if len(text) > 2:
            course_list.append(text[0:6].strip())

    service = Service("chromedriver.exe")
    driver = webdriver.Chrome(service=service)

    course_enrolment = {}

    for course in course_list:
        course_url = f"https://www.lse.ac.uk/resources/calendar{year}-{year+1}/courseGuides/{course[0:2]}/{year}_{course}.htm"
        driver.get(course_url)
        try:
            total_students = WebDriverWait(driver, 0.15).until(
                EC.presence_of_element_located((By.XPATH, "//div[@id='keyFacts-Content']/p[2]"))
            ).text
            department = course[0:2]
            course_enrolment[course] = (total_students.split(":")[-1].strip(), department)
            
        except:
            #print(f"cant find {course} in {year}")
            pass

    driver.quit()
    df = pd.DataFrame.from_dict(course_enrolment, orient="index", columns=["Enrolment", "Department"])
    return df
course_numbers_2024_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2024-2025/courseGuides/undergraduate.htm", 2024)
course_numbers_2023_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2023-2024/courseGuides/undergraduate.htm", 2023)
course_numbers_2022_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2022-2023/courseGuides/undergraduate.htm", 2022)
course_numbers_2021_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2021-2022/courseGuides/undergraduate.htm", 2021)
course_numbers_2020_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2020-2021/courseGuides/undergraduate.htm", 2020)
course_numbers_2019_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2019-2020/courseGuides/undergraduate.htm", 2019)
course_numbers_2018_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2018-2019/courseGuides/undergraduate.htm", 2018)
course_numbers_2017_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2017-2018/courseGuides/undergraduate.htm", 2017)
course_numbers_2016_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2016-2017/courseGuides/undergraduate.htm", 2016)

In [ ]:
#storing here as csv so u dont need to run code
course_numbers_2024_df.to_csv('data/course_numbers_2024.csv')
course_numbers_2023_df.to_csv('data/course_numbers_2023.csv')
course_numbers_2022_df.to_csv('data/course_numbers_2022.csv')
course_numbers_2021_df.to_csv('data/course_numbers_2021.csv')
course_numbers_2020_df.to_csv('data/course_numbers_2020.csv')
course_numbers_2019_df.to_csv('data/course_numbers_2019.csv')
course_numbers_2018_df.to_csv('data/course_numbers_2018.csv')
course_numbers_2017_df.to_csv('data/course_numbers_2017.csv')
course_numbers_2016_df.to_csv('data/course_numbers_2016.csv')

 ### for data cleaning, there are a lot of courses which have missing years. so maybe use only the courses where 4 or more recorded students numbers are present and use it like that

In [19]:
course_numbers_2024_df.head(5)


,Enrolment,Department
AC102,564,AC
AC103,256,AC
AC105,115,AC
AC106,115,AC
AC205,Unavailable,AC


# data from Tableu, containing number of students per year (all in one dict) - use for dropout rates. theres no department data for this. so for data cleaning i think u have to group each one into a department manually. basically we need this data but for department too but it doesnt exist

In [3]:
import tabula # you need to first install it
import pandas as pd
import warnings
warnings.simplefilter("ignore")
student_data_dict = {}
for year in [2016,2017,2018,2019,2020,2021,2022,2023]:
    dfs_in_list = tabula.read_pdf(f"data/student_data_{year}.pdf", pages="all")
    df = pd.concat(dfs_in_list, ignore_index=True)
    student_data_dict[year] = df

    

In [26]:
student_data_dict[2016].to_csv("Control.csv")

In [ ]:
## here i think its best to use SQL to take each df (stored as values) and then maybe make different tables and then combine them using sql

## data from HEFA - clean data already. shows how many staff members in all unis

In [148]:
staff_data_dict = {}
for year in [2016,2017,2018,2019,2020,2021,2022,2023]:
    df = pd.read_csv(f"data/staff_data_{year}.csv", skiprows=14)
    staff_data_dict[year] = df



In [164]:
staff_data_dict[2018]

In [ ]:
# extract lse from each df using sql and then make bar graph showing the trend or line plot

## data for each department offer rates

In [154]:

department_data_list_1 = tabula.read_pdf(f"data/department_offer_rates_16_to_19.pdf", pages='all',stream=True)
department_data_list_2 = tabula.read_pdf(f"data/department_offer_rates_20_to_23.pdf", pages='all',stream=True)
department_data_df_1 = department_data_list_1[0]
department_data_df_2 = department_data_list_2[0]
department_data_df_1.columns = [
    "Department", "Programme", 
    "2016_Apps", "2016_Offers", "2016_Entrants",
    "2017_Apps", "2017_Offers", "2017_Entrants",
    "2018_Apps", "2018_Offers", "2018_Entrants",
    "2019_Apps", "2019_Offers", "2019_Entrants"
]
department_data_df_2.columns = [
    "Department", "Programme", 
    "2020_Apps", "2020_Offers", "2020_Entrants",
    "2021_Apps", "2021_Offers", "2021_Entrants",
    "2022_Apps", "2022_Offers", "2022_Entrants",
    "2023_Apps", "2023_Offers", "2023_Entrants"
]


department_data_df = pd.merge(
    department_data_df_1,
    department_data_df_2,
    on=["Department", "Programme"],
    how="outer"
)
department_data_df = department_data_df.drop(index=2)
# Step 4: Preview result
department_data_df

,Department,Programme,2016_Apps,2016_Offers,2016_Entrants,2017_Apps,2017_Offers,2017_Entrants,2018_Apps,2018_Offers,...,2020_Entrants,2021_Apps,2021_Offers,2021_Entrants,2022_Apps,2022_Offers,2022_Entrants,2023_Apps,2023_Offers,2023_Entrants
0,Accounting UG Degree,NaN,"1,542",227,124,"1,540",253,135,"1,737",277,...,143,"1,881",176,115,"1,962",261,112,"1,929",265,115
1,Anthropology UG Degree,NaN,461,186,69,488,178,56,631,208,...,66,608,165,60,673,232,73,672,208,68
3,Economic History UG Degree,NaN,598,116,62,572,121,70,668,124,...,90,"1,015",104,71,981,130,71,918,130,69
4,Economics UG Degree,NaN,"2,731",489,220,"2,825",504,228,"2,895",543,...,265,"4,373",335,202,"4,213",437,261,"3,888",372,178
5,Finance UG Degree,NaN,576,87,34,572,79,37,599,125,...,68,882,91,53,"1,133",151,74,"1,058",123,65
6,Geography and Environment UG Degree,NaN,681,227,85,697,231,74,658,298,...,103,960,201,81,"1,083",242,92,"1,228",243,92
7,Government UG Degree,NaN,"1,832",377,158,"1,890",363,147,"2,194",415,...,184,"2,913",313,153,"2,660",417,183,"2,541",401,161
8,International History UG Degree,NaN,807,262,77,790,296,86,773,337,...,98,735,218,69,"1,235",400,131,"1,269",389,123
9,International Relations UG Degree,NaN,942,154,62,"1,014",164,64,910,163,...,87,849,100,44,778,158,77,767,141,43
10,Language Centre UG Degree,NaN,0,0,0,0,0,0,0,0,...,29,314,88,34,314,105,31,307,109,30


In [162]:
department_data_df.columns


Index(['Department', '2016_Apps', '2016_Offers', '2016_Entrants', '2017_Apps',
       '2017_Offers', '2017_Entrants', '2018_Apps', '2018_Offers',
       '2018_Entrants', '2019_Apps', '2019_Offers', '2019_Entrants',
       '2020_Apps', '2020_Offers', '2020_Entrants', '2021_Apps', '2021_Offers',
       '2021_Entrants', '2022_Apps', '2022_Offers', '2022_Entrants',
       '2023_Apps', '2023_Offers', '2023_Entrants'],
      dtype='object')

### farhaan needs to add api data from lse as well as word counts for the each word. do this for 2016,2019,2022 to see change over the years. Then we also do word counts for three other departments, maybe "lse stats", "lse economics" "lse law" to see if these departments all have the same trend or not

## complete data cleaning -dilan

In [7]:
def RemoveUnavailableCourses(dataframe):
    cleaned_df = dataframe[dataframe["Enrolment"] != "Unavailable"]
    return cleaned_df
    
course_numbers_2024_df = RemoveUnavailableCourses(course_numbers_2024_df)
course_numbers_2023_df = RemoveUnavailableCourses(course_numbers_2023_df)
course_numbers_2022_df = RemoveUnavailableCourses(course_numbers_2022_df)
course_numbers_2021_df = RemoveUnavailableCourses(course_numbers_2021_df)
course_numbers_2020_df = RemoveUnavailableCourses(course_numbers_2020_df)
course_numbers_2019_df = RemoveUnavailableCourses(course_numbers_2019_df)
course_numbers_2018_df = RemoveUnavailableCourses(course_numbers_2018_df)
course_numbers_2017_df = RemoveUnavailableCourses(course_numbers_2017_df)
course_numbers_2016_df = RemoveUnavailableCourses(course_numbers_2016_df)


NameError: name 'course_numbers_2024_df' is not defined

In [124]:
def clean_student_data(student_data):
    student_data_array = student_data.values.tolist()
    student_array = []
    x=3
    while x+1 < len(student_data_array):
        temp = []
        temp.append(student_data_array[x][0])
        first_year = 0
        second_year = 0
        final_year = 0
        total = 0
        if student_data_array[x][1] == "1st Year":
            first_year = student_data_array[x][2]
        elif student_data_array[x][1] == "2nd Year":
            second_year = student_data_array[x][2]
        elif student_data_array[x][1] == "Final Year":
            final_year = student_data_array[x][2]
        elif student_data_array[x][1] == "Total":
            total = student_data_array[x][2]
        x+=1
        while pd.isna(student_data_array[x][0]) and x < len(student_data_array)-1:
            if student_data_array[x][1] == "1st Year":
                first_year = student_data_array[x][2]
            elif student_data_array[x][1] == "2nd Year":
                second_year = student_data_array[x][2]
            elif student_data_array[x][1] == "Final Year":
                final_year = student_data_array[x][2]
            elif student_data_array[x][1] == "Total":
                total = student_data_array[x][2]
            x += 1
        temp.append(first_year)
        temp.append(second_year)
        temp.append(final_year)
        temp.append(total)
        student_array.append(temp)
    student_array.pop()
    return student_array

In [126]:
def create_df_student_data(student_data_array):
    column_names = ["Course name", "1st Year", "2nd Year", "Final Year","Total"]
    temp_df = pd.DataFrame(student_data_array,columns=column_names)
    return temp_df


In [128]:
student_data_2017 = student_data_dict[2017]
student_data_array_2017 = clean_student_data(student_data_2017)
missing_data_before = [["BSc in Accounting and Finance",136,134,114,384],["BSc in Actuarial Science",67,70,87,224],["BA in Anthropology and Law",14,15,16,45]]
missing_data_after = [["Sociology",49,33,36,118],["BSc in Statistics with Finance",3,22,21,46],["Total Undergraduate Degree Students",0,0,0,4769]]
student_data_array_2017 = missing_data_before + student_data_array_2017 + missing_data_after

In [130]:
student_data_2018 = student_data_dict[2018]
student_data_array_2018 = clean_student_data(student_data_2018)
missing_data_before = [["BSc in Accounting and Finance",147,126,138,411],["BSc in Actuarial Science",72,68,60,200],["BA in Anthropology and Law",16,18,15,49]]
missing_data_after = [["Sociology",38,47,31,116],["BSc in Statistics with Finance",0,3,17,20],["Total Undergraduate Degree Students",0,0,0,4992]]
student_data_array_2018 = missing_data_before + student_data_array_2018 + missing_data_after

In [132]:
student_data_2019 = student_data_dict[2019]
student_data_array_2019 = clean_student_data(student_data_2019)

missing_data_before = [["BSc in Accounting and Finance",116,152,117,385],["BSc in Actuarial Science",77,68,64,209],["BA in Anthropology and Law",16,15,17,48]]
missing_data_after = [["Sociology",37,40,46,123],["BSc in Statistics with Finance",0,0,4,4],["Total Undergraduate Degree Students",0,0,0,5097]]
student_data_array_2019 = missing_data_before + student_data_array_2019 + missing_data_after

In [134]:
student_data_2020 = student_data_dict[2020]
student_data_array_2020 = clean_student_data(student_data_2020)

missing_data_before = [["BSc in Accounting and Finance",144,120,148,412],["BSc in Actuarial Science",74,76,66,216],["BA in Anthropology and Law",27,16,15,58]]
missing_data_after = [["Sociology",58,39,36,133],["Total Undergraduate Degree Students",0,0,0,5559]]
student_data_array_2020 = missing_data_before + student_data_array_2020 + missing_data_after

In [136]:
student_data_2021 = student_data_dict[2021]
student_data_array_2021 = clean_student_data(student_data_2021)

missing_data_before = [["BSc in Accounting and Finance",116,150,121,387],["BSc in Actuarial Science",81,71,68,220],["BA in Anthropology and Law",21,29,15,65]]
missing_data_after = [["Sociology",53,59,40,152],["Total Undergraduate Degree Students",0,0,0,5485]]
student_data_array_2021 = missing_data_before + student_data_array_2021 + missing_data_after

In [138]:
student_data_2022 = student_data_dict[2022]
student_data_array_2022 = clean_student_data(student_data_2022)

missing_data_before = [["BSc in Accounting and Finance",113,122,139,374],["BSc in Actuarial Science",85,71,64,220],["BA in Anthropology and Law",18,22,25,65]]
missing_data_after = [["Sociology",51,50,56,157],["Total Undergraduate Degree Students",0,0,0,5862]]
student_data_array_2022 = missing_data_before + student_data_array_2022 + missing_data_after


In [140]:
student_data_2023 = student_data_dict[2023]
student_data_array_2023 = clean_student_data(student_data_2022)

missing_data_before = [["BSc in Accounting and Finance",118,113,117,348],["BSc in Actuarial Science",64,73,64,201],["BSc in Actuarial Science (with a Placement Year)",12,0,0,12],["BA in Anthropology and Law",19,20,20,59]]
missing_data_after = [["Sociology",37,51,50,138],["Total Undergraduate Degree Students",0,0,0,5639]]
student_data_array_2023 = missing_data_before + student_data_array_2023 + missing_data_after

In [146]:
student_data_2017_df = create_df_student_data(student_data_array_2017)
student_data_2018_df = create_df_student_data(student_data_array_2018)
student_data_2019_df = create_df_student_data(student_data_array_2019)
student_data_2020_df = create_df_student_data(student_data_array_2020)
student_data_2021_df = create_df_student_data(student_data_array_2021)
student_data_2022_df = create_df_student_data(student_data_array_2022)
student_data_2023_df = create_df_student_data(student_data_array_2023)
student_data_2020_df.head()

,Course name,1st Year,2nd Year,Final Year,Total
0,BSc in Accounting and Finance,144,120,148,412
1,BSc in Actuarial Science,74,76,66,216
2,BA in Anthropology and Law,27,16,15,58
3,BSc in Business Mathematics and Statistics,0,0,5,5
4,BSc in Econometrics and Mathematical Economics,0,0,14,14


In [160]:
department_data_df = department_data_df.drop("Programme", axis=1)
department_data_df.head()

,Department,2016_Apps,2016_Offers,2016_Entrants,2017_Apps,2017_Offers,2017_Entrants,2018_Apps,2018_Offers,2018_Entrants,...,2020_Entrants,2021_Apps,2021_Offers,2021_Entrants,2022_Apps,2022_Offers,2022_Entrants,2023_Apps,2023_Offers,2023_Entrants
0,Accounting UG Degree,"1,542",227,124,"1,540",253,135,"1,737",277,141,...,143,"1,881",176,115,"1,962",261,112,"1,929",265,115
1,Anthropology UG Degree,461,186,69,488,178,56,631,208,64,...,66,608,165,60,673,232,73,672,208,68
3,Economic History UG Degree,598,116,62,572,121,70,668,124,63,...,90,"1,015",104,71,981,130,71,918,130,69
4,Economics UG Degree,"2,731",489,220,"2,825",504,228,"2,895",543,249,...,265,"4,373",335,202,"4,213",437,261,"3,888",372,178
5,Finance UG Degree,576,87,34,572,79,37,599,125,52,...,68,882,91,53,"1,133",151,74,"1,058",123,65


Data we have:

* We have data from 2017-2023 for the course and students in each year
* We have data for departments applications, offers and entrants for 2016-2023
* We have data for module code and number of students studying a module
* We have staff data from 2016 to 2023

# varuns part

visualise data for ALL lse students over the years and see if there is an increase or decrease, then we can answer subquestions

 - To answer if "Are certain departments experiencing a change in student enrolment?" use cleaned data and visualise. maybe 2-3 plots. going to have to group programmes into departments, this is a bit long. but maybe theres another way of doing it idk
 - Then visualise the data that tells you how many people in each programme to explain the graph above. maybe stats had a big increase and maybe thats because ds became more popular in a certain year. maybe we can also explore why there was spiek using reddit api and wordcloud
 - 

"Is there a change in the number of applicants and offer rates from LSE"
- to answer this just visualise the data provided and see if offer rates have increased or maybe places

word cloud plus analysis displayed here for the chosen departments

- visualise staff data and compare to the total places offered each year and also look at drop out rates. drop out rates increasing in one year might mean more applicants offered a place in that department that year 